# edas 003 - lab 1 - python

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import style
style.use('dark_background')

data = pd.read_csv("./ArquivosLaboratorio/Arquivo_Treino.csv", delimiter=';')
data.describe()
data.count()

In [ ]:
data.head(10)

In [ ]:

credit_approved = data["Loan_Status"].value_counts().plot(kind='bar', title='Frequencia - Crédito Aprovado / Negado')

In [ ]:
credit_approved = data["Loan_Status"] == "Y" 
data[credit_approved].describe()

In [ ]:
data['ApplicantIncome'].hist(bins=50, figsize=(6,6))

In [ ]:
data.boxplot(column='ApplicantIncome', figsize=(8,8))

In [ ]:
data.boxplot(column='ApplicantIncome', figsize=(7,7), by='Education')

In [ ]:
data['LoanAmount'].hist(bins=50, figsize=(8,8))

In [ ]:
temp1 = data['Credit_History'].value_counts(ascending=True)
temp2 = data.pivot_table(values='Loan_Status',index=['Credit_History'],aggfunc=lambda x: x.map({'Y':1,'N':0}).mean())
print('Frequency Table for Credit History:')
print(temp1)
print(temp1[0])
print(temp1[1])
print(temp1.index[0])

print('\nProbability of getting loan for each Credit History class:')
print(temp2)

In [ ]:


from pylab import rcParams
rcParams['figure.figsize'] = 10, 6

fig, (ax1, ax2) = plt.subplots(ncols=2, nrows=1)

ax1.set_xlabel('Histórico de Crédito')
ax1.set_ylabel('Contagem de Clientes')
ax1.set_title('Clientes por Histórico')

temp2.plot(kind='bar', ax=ax1)

ax2.set_xlabel('Histórico de Crédito')
ax2.set_ylabel('Probabilidade de Empréstimo')
ax2.set_title('Probabilidade de Empréstimo por Histórico')

temp2.plot(kind='bar', ax=ax2)

In [ ]:
temp3 = pd.crosstab(data['Credit_History'],
data['Loan_Status'])
temp3.plot(kind='bar', stacked=True, color=['red','blue'],
grid=False)

In [ ]:
temp3 = pd.pivot_table(data, index=['Credit_History','Gender'], columns='Loan_Status', aggfunc='count')
temp3['Loan_ID'].plot(kind='bar', stacked=True, color=['red', 'blue'], grid=False)

In [ ]:
data.apply(lambda x: sum(x.isnull()),axis=0)

In [ ]:
data.boxplot(column='LoanAmount')

In [ ]:
data.boxplot(column='LoanAmount', by=['Education','Self_Employed'])

In [ ]:
data['Self_Employed'].value_counts()

In [ ]:
table = data.pivot_table(values='LoanAmount', index='Self_Employed', columns='Education', aggfunc=np.median)
table

In [ ]:
data['Self_Employed'].fillna('No',inplace=True)


In [ ]:
# Define a função que retorna o valor da tabela pivot
def fage(x):
 return table.loc[x['Self_Employed'], x['Education']] 

data['LoanAmount'].fillna(data[data['LoanAmount'].isnull()].apply(fage, axis=1), inplace=True)



In [ ]:
data.apply(lambda x: sum(x.isnull()),axis=0)

In [ ]:
data['LoanAmount'].hist(bins=20)


In [ ]:
data['LoanAmount_log'] = np.log(data['LoanAmount'])
data['LoanAmount_log'].hist(bins=20)

In [ ]:
data['TotalIncome'] = data['ApplicantIncome'] + data['CoapplicantIncome']
data['TotalIncome_log'] = np.log(data['TotalIncome'])
data['TotalIncome_log'].hist(bins=20)


In [ ]:
data['ApplicantIncome'].hist(bins=20)   

In [ ]:
data['TotalIncome'] = data['ApplicantIncome'] + data['CoapplicantIncome']
data['TotalIncome_log'] = np.log(data['TotalIncome'])
data['TotalIncome_log'].hist(bins=20)


In [ ]:
from sklearn.preprocessing import LabelEncoder

data['Gender'].fillna(data['Gender'].mode()[0], inplace=True)
data['Married'].fillna(data['Married'].mode()[0], inplace=True)
data['Dependents'].fillna(data['Dependents'].mode()[0],inplace=True)
data['Loan_Amount_Term'].fillna(data['Loan_Amount_Term'].mode()[0], inplace=True)
data['Credit_History'].fillna(data['Credit_History'].mode()[0],inplace=True)

var_mod =['Gender','Married','Dependents','Education','Self_Employed','Property_Area','Loan_Status']

le = LabelEncoder()

for i in var_mod: 
    data[i] = le.fit_transform(data[i])

data.dtypes



In [ ]:
data.apply(lambda x: sum(x.isnull()))

In [30]:
#Importa os modelos da biblioteca scikit learn:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics

#Função para fazer um modelo de classificação para avaliar performance

def classification_model(model, data, predictors, outcome):
  #Ajusta o modelo:
  model.fit(data[predictors],data[outcome])
  #Faz previsões nos dados de treino:
  predictions = model.predict(data[predictors])
  #Mostra a acurácia
  accuracy = metrics.accuracy_score(predictions,data[outcome])
  print("Acurácia : %s" % "{0:.3%}".format(accuracy))
  #Realiza validação cruzada k-fold com 5 folds
  kf = KFold(5,shuffle=False)
  error = []
  
  for train, test in kf.split(data):
    # Filtra dados de treino
    train_predictors = (data[predictors].iloc[train,:])
    # O alvo que estamos usando para treinar o algoritmo
    train_target =  data[outcome].iloc[train]
    # Treinando o algoritmo com previsores e alvo
    model.fit(train_predictors, train_target)
    #Grava erros de cada loop de validação cruzada
    error.append(model.score(data[predictors].iloc[test,:], data[outcome].iloc[test]))

    print("Score da Validação Cruzada : %s" %"{0:.3%}".format(np.mean(error)))
  
    #Acerta de novo o modelo para que possa se referir fora da função
    model.fit(data[predictors],data[outcome])



In [32]:
outcome_var = 'Loan_Status'
model = LogisticRegression()
predictor_var = ['Credit_History']
classification_model(model, data, predictor_var,outcome_var)

Acurácia : 80.945%
Score da Validação Cruzada : 80.488%
Score da Validação Cruzada : 78.455%
Score da Validação Cruzada : 79.133%
Score da Validação Cruzada : 80.691%
Score da Validação Cruzada : 80.946%


In [33]:
predictor_var = ['Credit_History','Education','Married','Self_Employed','Property_Area']
classification_model(model, data, predictor_var, outcome_var)

Acurácia : 80.945%
Score da Validação Cruzada : 80.488%
Score da Validação Cruzada : 78.455%
Score da Validação Cruzada : 79.133%
Score da Validação Cruzada : 80.691%
Score da Validação Cruzada : 80.946%


In [35]:
model = DecisionTreeClassifier()
predictor_var = ['Credit_History','Gender','Married','Education']
classification_model(model, data, predictor_var, outcome_var)

Acurácia : 80.945%
Score da Validação Cruzada : 80.488%
Score da Validação Cruzada : 78.455%
Score da Validação Cruzada : 79.133%
Score da Validação Cruzada : 80.691%
Score da Validação Cruzada : 80.946%


In [36]:
#Podemos tentar diferentes combinações de variáveis:
predictor_var = ['Credit_History','Loan_Amount_Term']
classification_model(model, data, predictor_var, outcome_var)

Acurácia : 81.270%
Score da Validação Cruzada : 78.862%
Score da Validação Cruzada : 76.829%
Score da Validação Cruzada : 78.320%
Score da Validação Cruzada : 80.081%
Score da Validação Cruzada : 80.295%


In [38]:
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,LoanAmount_log,TotalIncome,TotalIncome_log
0,LP001002,1,0,0,0,0,5849,0,130.0,360.0,1.0,2,1,4.867534,5849,8.674026
1,LP001003,1,1,1,0,0,4583,1508,128.0,360.0,1.0,0,0,4.852030,6091,8.714568
2,LP001005,1,1,0,0,1,3000,0,66.0,360.0,1.0,2,1,4.189655,3000,8.006368
3,LP001006,1,1,0,1,0,2583,2358,120.0,360.0,1.0,2,1,4.787492,4941,8.505323
4,LP001008,1,0,0,0,0,6000,0,141.0,360.0,1.0,2,1,4.948760,6000,8.699515


In [37]:
model = RandomForestClassifier(n_estimators=100)
predictor_var = ['Gender', 'Married', 'Dependents',
'Education', 'Self_Employed', 'Loan_Amount_Term',
'Credit_History', 'Property_Area',
'LoanAmount_log','TotalIncome_log']
classification_model(model, data, predictor_var,outcome_var)

Acurácia : 100.000%
Score da Validação Cruzada : 76.423%
Score da Validação Cruzada : 74.797%
Score da Validação Cruzada : 75.610%
Score da Validação Cruzada : 76.423%
Score da Validação Cruzada : 77.040%


In [39]:
#Criar uma série:
featimp = pd.Series(model.feature_importances_,
index=predictor_var).sort_values(ascending=False)
print(featimp)

Credit_History      0.272153
TotalIncome_log     0.263348
LoanAmount_log      0.227294
Dependents          0.052410
Property_Area       0.049762
Loan_Amount_Term    0.043421
Married             0.025875
Education           0.023937
Gender              0.021851
Self_Employed       0.019948
dtype: float64


In [41]:
model = RandomForestClassifier(n_estimators=25,
min_samples_split=25, max_depth=7, max_features=1)
predictor_var = ['TotalIncome_log','LoanAmount_log','Credit_History','Dependents','Property_Area']
classification_model(model, data, predictor_var,outcome_var)

Acurácia : 82.899%
Score da Validação Cruzada : 79.675%
Score da Validação Cruzada : 78.049%
Score da Validação Cruzada : 79.946%
Score da Validação Cruzada : 80.894%
Score da Validação Cruzada : 81.273%
